## Libraries

In [1]:
import os
import io
import datetime
import time
import requests
import ee
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from PIL import Image
import logging
from dotenv import load_dotenv

## Setup

In [2]:
load_dotenv()

api_key = os.environ["GEOAPIFY_API_KEY"]

In [3]:
PROJECT = 'ee-dhruv100yadavofficial'
# ee.Authenticate()
ee.Initialize(project=PROJECT)

## Basic Parameters

In [4]:
region_polygon = ee.Geometry.Rectangle([-99, 29, -97, 31])
surface_variables = [
    "temperature_2m_above_ground",
    "specific_humidity_2m_above_ground",
    "relative_humidity_2m_above_ground",
    "u_component_of_wind_10m_above_ground",
    "v_component_of_wind_10m_above_ground",
    "precipitable_water_entire_atmosphere"
]
forecast_hours = [0, 6, 12, 18]
start_date = datetime.date(2025, 1, 1)
end_date   = datetime.date(2025, 2, 28)


base_output_dir = "/Users/dhruvyadav/Desktop/Research/Manmeet Sir Research/AI NWS/Data Collection/Img_Data"
if not os.path.exists(base_output_dir):
    os.makedirs(base_output_dir)

### Logging

In [5]:
logging.basicConfig(
    level=logging.INFO,
    format='%(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler(os.path.join(base_output_dir, "collection.log"), mode='w')
    ]
)

## Helper Functions

In [6]:
def get_geoapify_basemap():
    """
    The center is specified as "lonlat:-98,30" (longitude, latitude).
    """

    global api_key

    url = f"https://maps.geoapify.com/v1/staticmap?style=osm-carto&width=512&height=512&center=lonlat:-98,30&zoom=6&showLogo=false&apiKey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.content
    else:
        raise Exception("Error retrieving Geoapify basemap: " + response.text)

def get_forecast_image(forecast_dt, variable):
    """
    Retrieves the forecast image for a given datetime and variable from NOAA/GFS0P25.
    """
    collection = ee.ImageCollection("NOAA/GFS0P25") \
                  .filterDate(forecast_dt, forecast_dt.advance(1, 'hour'))
    forecast_img = ee.Image(collection.first())
    return forecast_img.select(variable)

def get_dynamic_vis_params(forecast_img, variable):
    """
    Calculates dynamic min and max values for the forecast image over the region.
    If the computation fails, fallback to preset values.
    """
    reducer = ee.Reducer.minMax()
    stats = forecast_img.reduceRegion(reducer, region_polygon, scale=1000, maxPixels=1e9)
    min_val = stats.get(variable + "_min")
    max_val = stats.get(variable + "_max")
    try:
        min_val = float(min_val.getInfo())
        max_val = float(max_val.getInfo())
    except Exception as e:
        min_val = 0
        max_val = 1
    default_settings = {
        "temperature_2m_above_ground": {"palette": ['blue', 'cyan', 'green', 'yellow', 'red'], "opacity": 0.6},
        "specific_humidity_2m_above_ground": {"palette": ['white', 'blue'], "opacity": 0.6},
        "relative_humidity_2m_above_ground": {"palette": ['white', 'green'], "opacity": 0.6},
        "u_component_of_wind_10m_above_ground": {"palette": ['white', 'purple'], "opacity": 0.6},
        "v_component_of_wind_10m_above_ground": {"palette": ['white', 'orange'], "opacity": 0.6},
        "precipitable_water_entire_atmosphere": {"palette": ['white', 'blue'], "opacity": 0.6}
    }
    settings = default_settings.get(variable, {"palette": ['white'], "opacity": 1})
    return {"min": min_val, "max": max_val, "palette": settings["palette"], "opacity": settings["opacity"]}

def get_display_image(ee_img):
    """
    Retrieves a PNG thumbnail image for an Earth Engine image.
    """
    bounds = [-99, 29, -97, 31]
    thumb_params = {'region': bounds, 'dimensions': '512x512', 'format': 'png'}
    url = ee_img.getThumbURL(thumb_params)
    response = requests.get(url)
    if response.status_code == 200:
        return response.content
    else:
        raise Exception("Error retrieving thumbnail: " + response.text)

def composite_images(basemap_bytes, overlay_bytes):
    """
    Composites the overlay (with transparency) on top of the basemap.
    Both images are assumed to be 512x512.
    """
    base_img = Image.open(io.BytesIO(basemap_bytes)).convert("RGBA")
    overlay_img = Image.open(io.BytesIO(overlay_bytes)).convert("RGBA")
    composite = Image.alpha_composite(base_img, overlay_img)
    return composite

def crop_watermark(image, watermark_height=50):
    """
    Crops out the bottom 'watermark_height' pixels from the image.
    """
    width, height = image.size
    return image.crop((0, 0, width, height - watermark_height))

def resize_image(image, size=(512,512)):
    """
    Resizes the image to the specified size.
    """
    return image.resize(size, Image.Resampling.LANCZOS)

def plot_variable_subplots(variable, date, forecast_hours):
    """
    Creates a 2x2 subplot figure for one variable across the forecast hours for a given date.
    For each forecast hour, it:
      - Retrieves the forecast image and computes dynamic visualization parameters.
      - Visualizes the forecast overlay and retrieves it as a PNG.
      - Composites the overlay on the basemap (fetched once via Geoapify),
      - Crops out the watermark and resizes to 512x512.
    Then it arranges these four images in a 2x2 subplot, adds titles for each forecast hour,
    an overall title (the variable name), and a discrete colorbar using the same palette.
    Finally, the entire subplot figure is resized to 512x512 and returned as a PIL image.
    """
    basemap_bytes = get_geoapify_basemap()
    composite_images_list = []
    vis_params_list = []
    min_values = []
    max_values = []
    
    for hour in forecast_hours:
        forecast_dt = ee.Date(date.isoformat()).advance(hour, 'hour')
        forecast_img = get_forecast_image(forecast_dt, variable)
        vis_params = get_dynamic_vis_params(forecast_img, variable)
        vis_params_list.append(vis_params)
        min_values.append(vis_params['min'])
        max_values.append(vis_params['max'])
        
        forecast_overlay = forecast_img.visualize(**vis_params)
        overlay_bytes = get_display_image(forecast_overlay)
        comp_img = composite_images(basemap_bytes, overlay_bytes)
        comp_img = crop_watermark(comp_img, watermark_height=50)
        comp_img = resize_image(comp_img, size=(512,512))
        composite_images_list.append(comp_img)
    
    global_min = min(min_values)
    global_max = max(max_values)
    
    fig, axes = plt.subplots(2, 2, figsize=(8,8))
    axes = axes.flatten()
    
    for i, comp_img in enumerate(composite_images_list):
        ax = axes[i]
        ax.imshow(np.array(comp_img))
        ax.set_title(f"Forecast Hour: {forecast_hours[i]:02d}", fontsize=10)
        ax.axis("off")
    
    fig.suptitle(variable, fontsize=16)
    
    palette = vis_params_list[0]['palette']
    cmap = ListedColormap(palette)
    boundaries = np.linspace(global_min, global_max, len(palette)+1)
    norm = BoundaryNorm(boundaries, ncolors=len(palette))
    
    from matplotlib.cm import ScalarMappable
    sm = ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    fig.colorbar(sm, ax=axes.tolist(), fraction=0.03, pad=0.04).set_label(f"{variable} value", fontsize=12)
    
    temp_filename = "temp_subplot.png"
    plt.savefig(temp_filename, dpi=300, bbox_inches='tight')
    plt.close(fig)
    
    final_img = Image.open(temp_filename).convert("RGBA")
    final_img = resize_image(final_img, size=(512,512))
    os.remove(temp_filename)
    return final_img


## Main Loop

In [ ]:
current_date = start_date
while current_date <= end_date:

    # Directory Handling 
    
    month_dir = current_date.strftime("%b")
    month_path = os.path.join(base_output_dir, month_dir)
    if not os.path.exists(month_path):
        os.makedirs(month_path)
        logging.info(f"Created directory: {month_path}")
        
    date_dir = current_date.strftime("%Y-%m-%d")
    date_path = os.path.join(month_path, date_dir)
    if not os.path.exists(date_path):
        os.makedirs(date_path)
        logging.info(f"Created directory: {date_path}")
    
    for variable in surface_variables:
        try:
            final_subplot_img = plot_variable_subplots(variable, current_date, forecast_hours)
            out_filename = f"{variable}_subplots.png"
            save_path = os.path.join(date_path, out_filename)
            final_subplot_img.save(save_path)
            logging.info(f"Saved subplot for {current_date} variable {variable} at {save_path}")
        except Exception as e:
            logging.error(f"Error processing {current_date} variable {variable}: {e}")
    
    current_date += datetime.timedelta(days=1)

logging.info("Processing complete.")

Created directory: /Users/dhruvyadav/Desktop/Research/Manmeet Sir Research/AI NWS/Data Collection/Img_Data/Jan
Created directory: /Users/dhruvyadav/Desktop/Research/Manmeet Sir Research/AI NWS/Data Collection/Img_Data/Jan/2025-01-01
Saved subplot for 2025-01-01 variable temperature_2m_above_ground at /Users/dhruvyadav/Desktop/Research/Manmeet Sir Research/AI NWS/Data Collection/Img_Data/Jan/2025-01-01/temperature_2m_above_ground_subplots.png
Saved subplot for 2025-01-01 variable specific_humidity_2m_above_ground at /Users/dhruvyadav/Desktop/Research/Manmeet Sir Research/AI NWS/Data Collection/Img_Data/Jan/2025-01-01/specific_humidity_2m_above_ground_subplots.png
Saved subplot for 2025-01-01 variable relative_humidity_2m_above_ground at /Users/dhruvyadav/Desktop/Research/Manmeet Sir Research/AI NWS/Data Collection/Img_Data/Jan/2025-01-01/relative_humidity_2m_above_ground_subplots.png
Saved subplot for 2025-01-01 variable u_component_of_wind_10m_above_ground at /Users/dhruvyadav/Desktop/

_________

## Re-Running for 6 Feb 2025 (v_component_of_wind_10m_above_ground)

In [9]:
surface_variables = [
    "v_component_of_wind_10m_above_ground"
]

start_date = datetime.date(2025, 2, 6)
end_date   = datetime.date(2025, 2, 6)


In [10]:
# ---------------------------
# Main Loop: Process All Dates & Variables and Save to Google Drive
# ---------------------------
current_date = start_date
while current_date <= end_date:
    # Create a directory for the month in Google Drive.
    month_dir = current_date.strftime("%b")
    month_path = os.path.join(base_output_dir, month_dir)
    if not os.path.exists(month_path):
        os.makedirs(month_path)

    # Create a directory for the current date inside the month directory.
    date_dir = current_date.strftime("%Y-%m-%d")
    date_path = os.path.join(month_path, date_dir)
    if not os.path.exists(date_path):
        os.makedirs(date_path)

    for variable in surface_variables:
        try:
            # Create the subplot image for the given variable and date.
            final_subplot_img = plot_variable_subplots(variable, current_date, forecast_hours)
            # Define the output filename.
            out_filename = f"{variable}_subplots.png"
            save_path = os.path.join(date_path, out_filename)
            final_subplot_img.save(save_path)
            print(f"Saved subplot for {current_date} variable {variable} at {save_path}")
        except Exception as e:
            print(f"Error processing {current_date} variable {variable}: {e}")

    current_date += datetime.timedelta(days=1)


Saved subplot for 2025-02-06 variable v_component_of_wind_10m_above_ground at /content/drive/MyDrive/GEE_Subplots/Feb/2025-02-06/v_component_of_wind_10m_above_ground_subplots.png
